In [1]:
import os
import numpy as np
import pandas as pd
import re

batch = 10

#Pandas options to display max rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

PATH = os.getcwd()
FILE = f'Game_Data_{batch}.csv'

all_data = pd.read_csv(os.path.join(PATH, FILE), sep='\t', encoding='utf-8')

#Global functions
def gameResultMatrix(dataframe):
    #Get by player results
    by_player_results = dataframe.loc[:, ['Match_Result', 'Active_Player', 'Active_Enemy']].groupby(['Active_Player','Match_Result']).count()
    by_player_results.reset_index(inplace=True, drop=False)
    by_player_results['Pct_Total'] = [round(i/len(dataframe),5) for i in by_player_results.Active_Enemy.tolist()]
    
    #Sort results by player
    bpr = by_player_results.sort_values(by=['Match_Result', 'Pct_Total'], ascending=False)
    match_values = list(set(bpr.Match_Result.tolist())) #Unique list of match result values e.g. Win, Loss, Draw
    all_matchs = [bpr.query('Match_Result == @value') for value in match_values] #Split dataframe into multiple dfs
    for i in range(0, len(all_matchs)):
        all_matchs[i].columns = ['Player', match_values[i], f'{match_values[i]}_Count', f'{match_values[i]}_Pct_Total'] #Remap column names for every df in all_matchs
    characters = list(set(bpr.Active_Player.tolist())) #Get unique list of characters
    df_characters = pd.DataFrame(characters, columns=['Player']) #Instantiate DataFrame with players as first column
    for df in all_matchs:
        df_characters = df_characters.merge(df, how='left', left_on='Player', right_on='Player') #Merge all DataFrames together
    
    #Check win-to-loss ratio of all player characters
    df_raw_nums = df_characters.loc[:, ['Player', 'Win_Count', 'Loss_Count','Draw_Count']]
    df_raw_nums.fillna(0, inplace=True)
    df_raw_nums['Loss_Count'] = df_raw_nums['Loss_Count'].astype(int)
    df_raw_nums['Draw_Count'] = df_raw_nums['Draw_Count'].astype(int)
    df_raw_nums['Win_Loss_Ratio'] = [round(df_raw_nums.Win_Count[idx]/df_raw_nums.Loss_Count[idx],1) if df_raw_nums.Loss_Count[idx] != 0 else 0 for idx in range(0, len(df_characters))]
    df_raw_nums['Win_Draw_Ratio'] = [round(df_raw_nums.Win_Count[idx]/df_raw_nums.Draw_Count[idx],1) if df_raw_nums.Draw_Count[idx] != 0 else 0 for idx in range(0, len(df_characters))]
    df_raw_nums['Prevalence'] = [len(dataframe)//len(dataframe.query(f'Active_Player == "{idx}"')) for idx in df_raw_nums.Player]
    return df_raw_nums.sort_values(by='Win_Count', ascending=False), df_raw_nums.sort_values(by='Win_Loss_Ratio', ascending=False)

def PVPMatrix(dataframe):
    #Create tuples of faceoffs between two particular sets of characters
    dataframe['Faceoff'] = [(dataframe.Active_Player[idx], dataframe.Active_Enemy[idx]) for idx in range(0, len(dataframe))]
    faceoff = dataframe.loc[:, ['Faceoff', 'Match_Result', 'AP_HP']].copy() #Filter data to isolate faceoff results

    #Group results by particular faceoffs
    fo_results = faceoff.groupby(['Faceoff', 'Match_Result']).count()
    fo_results.reset_index(inplace=True, drop=False)

    #Reorder results into matrix with Win, Loss, and Draw columns
    fo_matrix = pd.DataFrame(None, index=[str(i) for i in list(set(fo_results.Faceoff.tolist()))], columns=['Win', 'Loss', 'Draw'])

    for row in fo_matrix.index.tolist():
        for idx in range(0, len(fo_results)): #Loop through index of DataFrame that is the source of the values
            if str(fo_results.Faceoff[idx]) == row:
                fo_matrix.loc[row, fo_results.Match_Result[idx]] = fo_results.AP_HP[idx]
            else:
                pass

    fo_matrix.fillna(0, inplace=True)
    fo_matrix['Win_Loss_Ratio'] = [round(fo_matrix.Win[idx]/fo_matrix.Loss[idx],1) if fo_matrix.Loss[idx] != 0 else 0 for idx in range(0, len(fo_matrix))]
    fo_matrix = fo_matrix.sort_values(by='Win_Loss_Ratio', ascending=False)
    fo_matrix['Win_Draw_Ratio'] = [round(fo_matrix.Win[idx]/fo_matrix.Draw[idx],1) if fo_matrix.Draw[idx] != 0 else 0 for idx in range(0, len(fo_matrix))]
    fo_matrix['Pairings'] = [str(set(sorted(eval(idx)))) for idx in fo_matrix.index.tolist()] #Add new column with normalized names for aggregation
    fm2 = fo_matrix.loc[:,['Pairings','Win', 'Loss', 'Draw', 'Win_Loss_Ratio', 'Win_Draw_Ratio']].groupby('Pairings').sum()
    fm2 = fm2.sort_values(by=['Win_Loss_Ratio'], ascending=False)
    return fm2

def AutoAnalyse(dataframe, batch_num=0):

    #Get overall match results
    batch = batch_num #Change every time this thing runs
    match_results = dataframe.loc[:, ['Match_Result', 'AP_SPD']].groupby(['Match_Result']).count()
    match_results.reset_index(inplace=True, drop=False)
    match_results['Pct_Total'] = [round(i/len(dataframe),2) for i in match_results.AP_SPD.tolist()]
    match_results['Batch'] = batch 
    game_by_win_count, game_by_winlossratio = gameResultMatrix(dataframe)
    pvp_results = PVPMatrix(dataframe)

    #Output all results to files for future analysis
    match_results.to_csv(f'Match_Batch_{str(batch)}.csv', index=False, sep='\t', encoding='utf-8')
    dataframe.to_csv(f'Batch_{str(batch)}.csv', index=False, sep='\t', encoding='utf-8')
    game_by_win_count.to_csv(f'Results_Batch_{str(batch)}.csv', index=False, sep='\t', encoding='utf-8')
    pvp_results.to_csv(f'PVP_Batch_{str(batch)}.csv', index=False, sep='\t', encoding='utf-8')

    return pvp_results, game_by_win_count

pvp_, win_ = AutoAnalyse(all_data, batch_num=batch)
print(f'Rows {len(all_data)}')
win_.sort_values(by='Win_Loss_Ratio', ascending=False)

Rows 151290


,Player,Win_Count,Loss_Count,Draw_Count,Win_Loss_Ratio,Win_Draw_Ratio,Prevalence
10,Sal,1378,2,4489,689.0,0.3,25
13,Stevie B,1899,4,1379,474.8,1.4,46
15,Mauser,1403,3,4548,467.7,0.3,25
8,Vladimir Von Vlak,1987,5,1354,397.4,1.5,45
12,Svetlana,1295,4,4796,323.8,0.3,24
1,Paramilitary,1284,4,4695,321.0,0.3,25
0,Earthquake,1343,6,4671,223.8,0.3,25
16,Alien Hunter,1454,9,4515,161.6,0.3,25
9,Corporate Muscle,1106,8,4700,138.2,0.2,26
3,DYS-1000,3032,25,467,121.3,6.5,42


In [2]:
df_test = [i//2 for i in np.array(win_.iloc[0:4,1:])]
dft2 = pd.DataFrame(df_test, index = win_.Player[0:4].index)
dft2 = pd.concat([win_.Player[0:4],dft2],axis=1, ignore_index=True)
dft2.columns=win_.columns
win_alt = pd.concat([dft2,win_.iloc[4:, :]], axis=0)
win_alt = win_alt.sort_values(by='Win_Loss_Ratio', ascending=False)
win_alt

,Player,Win_Count,Loss_Count,Draw_Count,Win_Loss_Ratio,Win_Draw_Ratio,Prevalence
10,Sal,1378.0,2.0,4489.0,689.0,0.3,25.0
13,Stevie B,1899.0,4.0,1379.0,474.8,1.4,46.0
15,Mauser,1403.0,3.0,4548.0,467.7,0.3,25.0
8,Vladimir Von Vlak,1987.0,5.0,1354.0,397.4,1.5,45.0
12,Svetlana,1295.0,4.0,4796.0,323.8,0.3,24.0
1,Paramilitary,1284.0,4.0,4695.0,321.0,0.3,25.0
0,Earthquake,1343.0,6.0,4671.0,223.8,0.3,25.0
16,Alien Hunter,1454.0,9.0,4515.0,161.6,0.3,25.0
9,Corporate Muscle,1106.0,8.0,4700.0,138.2,0.2,26.0
3,DYS-1000,3032.0,25.0,467.0,121.3,6.5,42.0


#Batch_5
Cerulean Crusader still has highest Win_Loss_Ratio by a factor of EIGHT. Doug has a slightly higher adjusted Prevalence than the other three characters (4 vs 3). Prevalence for all five bosses increased by a factor of TEN. Still at least TWENTY TIMES lower than other NPCs.

#Batch_6
Prevalence for bosses increased by about ten percent. Cerulean Crusader's high Win_Loss_Ratio might have to do with its comparatively low prevalence. Cards account for roughly 26% of outcome variance. Cards were played in 48% of encounters.

#Batch_7
No discernable change in output save for Stevie B not losing (all other bosses remain undefeated. "Nice night for a walk" and "Let off some steam" still account for 5% and 4% of Wins respectively.

#Batch_8
Prevalence for Bosses now approx. 100:1. Cerulean crusader now within norms.

In [ ]:
#Evaluate cards distribution
card_eval = all_data.loc[:, ['Played_Cards','Match_Result']].copy()
card_eval['Card'] = ['No Card' if eval(i)[0] == None else eval(i)[0] for i in card_eval.Played_Cards]
card_eval = card_eval.loc[:, ['Card', 'Match_Result', 'Played_Cards']]
card_ct = card_eval.groupby(['Card', 'Match_Result']).count()
card_ct.reset_index(inplace=True, drop=False)
card_ct['Pct_Total'] = [round(i/len(card_eval),3) for i in card_ct.Played_Cards]
card_ct = card_ct.sort_values(by=['Match_Result','Pct_Total'], ascending=False)
card_ct

In [ ]:
#Evaluate weapon distribution
wpn_eval = all_data.loc[:, ['AP_WPN','Match_Result']].copy()
wpn_eval['Weapon'] = [i if type(i) == str else 'None' for i in wpn_eval.AP_WPN]
wpn_eval = wpn_eval.loc[:, ['Weapon', 'Match_Result', 'AP_WPN']]
wpn_ct = wpn_eval.groupby(['Weapon', 'Match_Result']).count()
wpn_ct.reset_index(inplace=True, drop=False)
wpn_ct['Pct_Total'] = [round(i/len(wpn_eval),3) for i in wpn_ct.AP_WPN]
wpn_ct = wpn_ct.sort_values(by=['Match_Result','Pct_Total'], ascending=False)
wpn_ct